# Word Embeddings

Los Word Embeddings son aquellas técnicas y modelos de lenguaje que permiten mapear palabras a vectores de valores continuos. Durante el entrenamiento de dichos vectores se buscará que capturen la información semántica de las palabras.

<img src=https://miro.medium.com/max/1280/1*OEmWDt4eztOcm5pr2QbxfA.png widt=500px>

Gracias a que los vectores tienen información sobre la semántica, mediante operaciones vectoriales podemos encontrar palabras (o documentos) que tienen un significado similar.

La idea principal de este tipo de modelos es que **palabras que aparecen en contextos similares tienen semánticas similares**. **Concepto de sustituibilidad**.

**Aquellas palabras que semánticamente son similares tendrán - idealmente - vectores-palabra cercanos entre sí.**

Existen multitud de modelos, en esta sesión veremos solo algunas. 

## Word2Vec

El modelo más famoso de word embeddings. Inventado por [Tomas Mikolov en 2013 en Google](https://arxiv.org/pdf/1310.4546.pdf)

https://radimrehurek.com/gensim/models/word2vec.html

### Hyperparameters

Algunos de los hiperparámetros que deberemos tener en cuenta y configuraremos:

- size: dimensionalidad de las palabras vector
- window: ventana para obtener el contexto de cada palabra. Se mide en número de palabras máximo entre la palabra actual y la palabra a predecir
- min_count: frecuencia mínima de aparición de una palabra para que sea considerada en el entrenamiento
- sg: algoritmo escogido. 1 para Skip-Gram, 0 para CBOW
- hs: si = 1, se utiliza una softmax jerárquica. Si = 0, y negative != 0, se emplea negative sampling
- negative: si = 0, no se usa negative sampling. Si es > 0, se usará negative sampling. El valor indica el número de "palabras ruidosas" se incluirán (usual entre 5-20 para datasets pequeños, entre 2-5 para datasets grandes)

### Noción de contexto

<img src=https://docs.chainer.org/en/v4.0.0b2/_images/center_context_word.png width=450px>

## Atributos

- wv: word vectors, contiene el mapeo entre palabras y vectores (embeddings)
- vocabulary: vocabulario (o diccionario) del modelo


## Negative sampling

Cuando se trabaja en NLP el tamaño del vocabulario suele tener una cardinalidad enorme. Esto afecta a los modelos de lenguaje a la hora de predecir aquellas palabras que, aunque correctas, no son demasiado frecuentes.

Además, contextos muy comunes (como los que podrían ser aquellos en los que se encuentran muchas stop words) hacen que el entrenamiento sea lento. Se emplea, por tanto, para reducir la carga computacional al problema.

La solución que se propone - e implementa - Word2Vec es que cada palabra tenga una determinada probabilidad de ser eliminada del training set. Dicha probabilidad estará relacionada con la frecuencia de repetición de dicha palabra.


## Arquitecturas

Existen dos arquitecturas de este modelo: CBOW y Skip Gram.


#### CBOW (Continuous Bag of Words)

Durante el entrenamiento, el modelo tratará de **predecir la palabra actual** dado el contexto en el que se encuentre. La capa de entrada contendrá las palabras-contexto y la de salida será la palabra actual (o palabra a predecir). La capa intermedia tendrá una dimension igual al número de dimensiones en el que queremos representar la palabra actual a la salida.

<img src=https://miro.medium.com/max/1104/0*CCsrTAjN80MqswXG width=400px>


#### Skip Gram

Durante el entrenamiento, el modelo tratará de predecir **el contexto (palabras-contexto)** a una palabra dada. La capa de entrada contendrá la palabra actual y la de salida serán las palabras contexto. La capa intermedia es análoga a la presente en la arquitectura CBOW.

<img src=https://miro.medium.com/max/1280/0*Ta3qx5CQsrJloyCA.png width=300px>

### ¿Cuál es mejor?

En general, depende. CBOW, al haber sido entrenado para predecir una palabra dado un contexto, será algo mejor _rellenando huecos_, aunque eso puede significar que palabras correctas pero menos comunes no aparezcan como resultado algunas veces. Skip Gram, en cambio, debería ser mejor infiriendo relaciones más concretas en contextos similares (por ejemplo, "me gusta el color verde" y "me encanta el color azul", y la diferencia en la intensidad del sentimiento expresado).

Si atendemos a los comentarios de Mikolov:

- Skip-gram: funciona bien con conjuntos de datos pequeños, representando bien incluso palabras o frases extrañas (poco comunes)
- CBOW: entrenamiento varias veces más rápido, su performance mejor para aquellas palabras más frecuentes que el resto

## Palabras más similares

In [2]:
def print_sim_words(word, model1, model2):
    query = "Most similar to {}".format(word) 
    print(query)
    print("-"*len(query))
    for (sim1, sim2) in zip(model1.wv.most_similar(word), model2.wv.most_similar(word)):
        print("{}:{}{:.3f}{}{}:{}{:.3f}".format(sim1[0],
                                               " "*(20-len(sim1[0])), 
                                               sim1[1], 
                                               " "*10, 
                                               sim2[0],
                                               " "*(20-len(sim2[0])),
                                               sim2[1]))
    print("\n")

## Importamos las librerías

In [3]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

## Lectura de datos

In [4]:
corpus = LineSentence('../../datasets/spanish_news_corpus_doc.txt', limit=200)

## Hyperparameters

In [5]:
sg_params = {
    'sg': 1,
    'size': 300,
    'min_count': 5,
    'window': 5,
    'hs': 0,
    'negative': 20,
    'workers': 4
}

cbow_params = {
    'sg': 0,
    'size': 300,
    'min_count': 5,
    'window': 5,
    'hs': 0,
    'negative': 20,
    'workers': 4
}

## Inicializamos los objetos Word2Vec

In [6]:
# Skip Gram
w2v_sg = Word2Vec(**sg_params)

# CBOW
w2v_cbow = Word2Vec(**cbow_params)

## Construímos el vocabulario

In [7]:
# Skip Gram
w2v_sg.build_vocab(corpus)

# CBOW
w2v_cbow.build_vocab(corpus)

In [8]:
print('Vocabulario compuesto por {} palabras'.format(len(w2v_sg.wv.vocab)))

Vocabulario compuesto por 2576 palabras


In [10]:
print('Vocabulario compuesto por {} palabras'.format(len(w2v_cbow.wv.vocab)))

Vocabulario compuesto por 2576 palabras


## Entrenamos los pesos de los embeddings

In [11]:
# Skip Gram
w2v_sg.train(sentences=corpus, total_examples=w2v_sg.corpus_count, epochs=40)

(2047796, 3906600)

In [12]:
# CBOW
w2v_cbow.train(sentences=corpus, total_examples=w2v_cbow.corpus_count, epochs=40)

(2048520, 3906600)

## Guardamos los modelos

In [ ]:
w2v_sg.save('../../data/w2v_sg_d300_mc5_w5.pkl')
w2v_cbow.save('../../data/w2v_cbow_d300_mc5_w5.pkl')

## Algunos resultados

In [34]:
print_sim_words('elecciones', w2v_cbow, w2v_sg)
print_sim_words('botín', w2v_cbow, w2v_sg)
print_sim_words('sánchez', w2v_cbow, w2v_sg)
print_sim_words('impeachment', w2v_cbow, w2v_sg)

Most similar to elecciones
--------------------------
generales:           0.833          generales:           0.747
forzar:              0.747          forzar:              0.644
cs:                  0.708          cs:                  0.598
unidas:              0.687          johnson:             0.564
próximas:            0.684          próximas:            0.556
cuentas:             0.682          sumar:               0.546
funciones:           0.680          apoyos:              0.540
sánchez:             0.680          moción:              0.536
apoyos:              0.670          pedir:               0.532
presentar:           0.668          rechaza:             0.525


Most similar to botín
---------------------
concurso:            0.890          ana:                 0.805
endesa:              0.865          jesús:               0.623
abascal:             0.845          presidenta:          0.609
álvarez:             0.844          álvarez:             0.607
vicepresidente:   

## Palabras fuera del vocabulario (OOV Words)

Los embeddings calculados a nivel de palabra no devolverán un vector para aquellos tokens que no hayan guardado en su vocabulario. Una vez que los vectores palabra han sido aprendidos, aquellas palabras que no han sido aprendidas durante el entrenamiento no tendrán representación.

In [27]:
'asereje' in w2v_cbow.wv.vocab

False

In [28]:
w2v_cbow.wv.most_similar('asereje')

KeyError: "word 'asereje' not in vocabulary"

Algunas estrategias para lidiar con palabras OOV:
- Asignar un vector que siga una distribución aleatoria uniforme. P. ej.:
`unk = np.random.uniform(-np.var(w2v.wv.vectors), np.var(w2v.wv.vectors), w2v.wv.vector_size)`
- Reemplazar por un token especial, conocido y distinto del resto, (`<unk>`) y entrenar los embeddings
- Reemplazar por un token especial, conocido y disinto del resto, y añadir información extra. P. ej.: `<unk_noun>` o `<unk_verb>`
- Utilizar modelos que no sean a nivel de palabra

## Visualización: Bonus

[Enlace](https://anvaka.github.io/pm/#/galaxy/word2vec-wiki?cx=-16179&cy=-1641&cz=4313&lx=0.3194&ly=-0.5230&lz=-0.4110&lw=0.6749&ml=300&s=1.75&l=1&v=d50_clean)

<img src=https://empresas.blogthinkbig.com/wp-content/uploads/2019/06/embeddings_galaxy.png width=650px>